### Student Information
Name: 陳宥任

Student ID: 111071603

GitHub ID: Kris0214

Kaggle name:

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the DM2023-Lab2-master. You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/t/09b1d0f3f8584d06848252277cb535f2) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/09b1d0f3f8584d06848252277cb535f2. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 27th 11:59 pm, Wednesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 31th 11:59 pm, Sunday)__. 

In [2]:
### Begin Assignment Here

# The first part please refer to file DM2023-Lab2-Master.ipynb.
# The Kaggle part was missed due to a missed upload deadline. This part has no results.

In [ ]:
# The following is the Model for the E.sun credit card competition.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import lightgbm as lgb
import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from optuna.visualization import plot_param_importances

# Training data sampling
''' 
Due to the large volume of training data provided in the competition,
 considering the limited hardware performance and time constraints,
 we opt to employ a random sampling approach to extract a certain quantity of training data for model training.
'''
df_train = pd.read_csv('training.csv')
df_public = pd.read_csv("public.csv")
df_full = pd.concat([df_train,df_public],axis=0)

np.random.seed(42)
total_size = df_full.shape[0]
sample_size = 4000000
sample_indices = np.random.choice(total_size, size=sample_size, replace=False)
sample_data = df_full.iloc[sample_indices]
sample_data.to_csv('training_sample.csv')


df_train = pd.read_csv('training_sample.csv')
print(df_train.shape)
df_train = df_train.drop('Unnamed: 0',axis=1)
df_test = pd.read_csv("public_processed.csv")

In [ ]:
'''
In the data preprocessing stage, we performed feature engineering by subdividing some time-related data into multiple features. 

For handling missing values, we experimented with several label encoding methods to impute the missing values,
and one of these approaches is presented here. 
 
Dealing with imbalanced data posed a significant challenge throughout the competition. 
We explored various methods,including random sampling for oversampling and undersampling, SMOTE, Borderline SMOTE, etc. 
The following outlines the approach involving random sampling for oversampling and undersampling.
'''
# Preprocessing
def preprocessing(df, bool_col, category_col):
    df['weekday'] = df['locdt'] % 7
    df['loctm_hh'] = df['loctm'].apply(lambda x: math.floor(x/10000))
    df['loctm_mm'] = df['loctm'].apply(lambda x: math.floor(x/100)-math.floor(x/10000)*100)
    df['loctm_ss'] = df['loctm'].apply(lambda x: math.floor(x)-math.floor(x/100)*100)
    df = df.drop(['locdt','loctm'], axis=1)
    
    for col in bool_col:
        df[col] = df[col].astype(bool)  
    for col in category_col:
        df[col] = df[col].astype('category')

    return df
    
col_convert_bool = ['ecfg','insfg','bnsfg','ovrlt','flbmk','flg_3dsmk']
col_convert_category = ['txkey','chid','cano','contp','etymd','mchno','acqic','mcc','stocn','scity','stscd','hcefg','csmcu']

df_train = preprocessing(df=df_train,bool_col=col_convert_bool,category_col=col_convert_category)
df_test = preprocessing(df=df_test,bool_col=col_convert_bool,category_col=col_convert_category)

# missing_value
def missing_value(df,carry_method = 'c'):
    df = df.drop('stscd',axis=1)
    #df.dropna(subset=['mcc', 'stocn'], inplace=True)
  
    df_missing = df[df.isnull().any(axis=1)]
    df_no_missing = df.dropna()

    model = LinearRegression()

    for feature in df.columns[df.isnull().any()]:
        y_columns = ['contp','conam','ecfg','insfg','iterm','bnsfg','flam1','ovrlt','flbmk','csmam','flg_3dsmk','weekday','loctm_hh','loctm_mm','loctm_ss']
        model.fit(df_no_missing[y_columns], df_no_missing[feature])
        predictions = model.predict(df_missing[y_columns])
        if carry_method == 'r':
            df_missing.loc[:, feature] = np.round(predictions)
        elif carry_method == 'c':
            df_missing.loc[:, feature] = np.ceil(predictions)
        elif carry_method == 'f':
            df_missing.loc[:, feature] = np.floor(predictions)
    df_filled = pd.concat([df_no_missing, df_missing])
    return df_filled

df_train = missing_value(df=df_train)
df_test = missing_value(df=df_test)

# resampling
def balanced_data(df, target_samples = 10000, r = 42):
    df_majority = df[df['label'] == 0]
    df_minority = df[df['label'] == 1]

    oversample = resample(df_minority, replace=True, n_samples=target_samples, random_state=r)

    undersample = resample(df_majority, replace=False, n_samples=target_samples, random_state=r)

    df_resampled = pd.concat([undersample, oversample])

    return df_resampled

df_train = balanced_data(df_train)
df_test['csmcu'] = df_test['csmcu'].astype('category')

In [ ]:
'''
In the model training phase, our team utilized LightGBM to train the model. 
We employed K-fold cross-validation to split the training set, and the evaluation metric was F1-score. 
Finally, we used the Optuna module to fine-tune hyperparameters, trained the model, and obtained the final prediction results
'''
# Model
X = df_train.drop('label', axis=1)
y = df_train['label']

trials = 10
K = 10

kfold = KFold(n_splits=K, shuffle=True, random_state=42)
categorical_features = ['chid', 'cano', 'contp', 'etymd', 'mchno', 'acqic', 'mcc', 'stocn', 'scity', 'hcefg', 'csmcu']

def objective(trial):
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    f1_scores = []
    
    for train_idx, val_idx in kfold.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        train_data = lgb.Dataset(X_train, label=y_train,categorical_feature=categorical_features)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
        
        model = lgb.train(params, train_data, valid_sets=[val_data])

        y_pred = model.predict(X_val, num_iteration=model.best_iteration)
        y_pred_binary = np.round(y_pred)

        f1 = f1_score(y_val, y_pred_binary)
        f1_scores.append(f1)
    return np.mean(f1_scores)

study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials = trials)
best_params = study.best_params

fig = optuna.visualization.plot_param_importances(study)
fig.show()

train_data = lgb.Dataset(X, label=y,categorical_feature=categorical_features)
best_model = lgb.train(best_params, train_data)
lgb.plot_importance(best_model)
plt.title("Features importance")
plt.show()

test_predictions = best_model.predict(df_test,categorical_feature=categorical_features)
df_submission = pd.read_csv("test_submission.csv")

df_submission['pred'] = np.where(test_predictions < 0.5, 0, 1)
df_submission['pred'] = df_submission['pred'].astype(int)
df_submission['txkey'] = df_submission['txkey'].astype(str)
df_submission['pred'] = np.abs(df_submission['pred'])
df_submission.to_csv('lightGBM_v1_20231115.csv',index=False)
print(best_params)
print(df_submission['pred'].value_counts())